 <picture>
 <source media="(prefers-color-scheme: dark)" srcset="_images/astroquery_project_logo_dark.svg">
 <source media="(prefers-color-scheme: light)" srcset="_images/astroquery_project_logo.svg">
 <img  class= "dark-light" alt="" style="float:right;height:50px;padding-right:20px;">
</picture>


# Astroquery Examples

Astroquery ( [https://astroquery.readthedocs.io](https://astroquery.readthedocs.io) )
is a set of tools for querying astronomical web forms and databases. It is a coordinated package
of the Astropy project ( [http://www.astropy.org](http://www.astropy.org/) ).

General imports:

In [4]:
from astropy import coordinates
import astropy.units as u

from astroquery.simbad import Simbad
from astroquery.vizier import Vizier

## SIMBAD queries

SIMBAD allows for simple queries based on coordinates or object names.

### Query by target ID

Use `query_object()` to query for a single object ID.

In [23]:
result_table = Simbad.query_object("M57")
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 57,18 53 35.0967,+33 01 44.883


You can get a list of object IDs with `quey_objectids()`.

In [25]:
IDs = Simbad.query_objectids("Ring Nebula")
IDs.show_in_notebook(display_length=10)

idx,ID
0,Gaia DR3 2090486618786534784
1,PLX 4377
2,BD+32 3246
3,CSI+32-18517
4,GCRV 11366
5,GSC2 N0223131306
6,HD 175353
7,IRAS 18517+3257
8,M 57
9,NGC 6720


You can query a list of objects with `query_objects()`. This is more efficient than repeated requests for single objects. There are rate limits that may vary, but you should not submit more than ~5-10 queries per second. A list is treated as a single request and will not result in overtaxing the server.

In [3]:
result_table = Simbad.query_objects(["M1", "M2", "M3", "M4"])
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 1,05 34 30.9,+22 00 53
1,M 2,21 33 27.02,-00 49 23.7
2,M 3,13 42 11.62,+28 22 38.2
3,M 4,16 23 35.22,-26 31 32.7


Wildcards and ranges can be used for identifiers.

`*` : Any string of characters (including an empty one)

`?` : Any character (exactly one character)

`[abc]` : Exactly one character taken from the list.

`[A-Z]` : Can also be defined by a range of characters.

`[^0-9]` : Any (one) character not in the list.

In [4]:
result_table = Simbad.query_object("m [1-9]", wildcard=True)
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 1,05 34 30.9,+22 00 53
1,M 2,21 33 27.02,-00 49 23.7
2,M 3,13 42 11.62,+28 22 38.2
3,M 4,16 23 35.22,-26 31 32.7
4,M 5,15 18 33.22,+02 04 51.7
5,NGC 6405,17 40 16.6,-32 14 31
6,NGC 6475,17 53 47.3,-34 50 28
7,M 8,18 03 37,-24 23.2
8,M 9,17 19 11.78,-18 30 58.5


Generating a list of targets programmatically may be easier and more reliable than wildcard searches.

In [5]:
targets = [f'M{_}'for _ in range(1,11)]; targets

['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10']

In [6]:
result_table = Simbad.query_objects(targets)
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 1,05 34 30.9,+22 00 53
1,M 2,21 33 27.02,-00 49 23.7
2,M 3,13 42 11.62,+28 22 38.2
3,M 4,16 23 35.22,-26 31 32.7
4,M 5,15 18 33.22,+02 04 51.7
5,NGC 6405,17 40 16.6,-32 14 31
6,NGC 6475,17 53 47.3,-34 50 28
7,M 8,18 03 37,-24 23.2
8,M 9,17 19 11.78,-18 30 58.5


### Query by region

You can query all objects in a region with `query_region()`. 
Use a coordinate or object ID and a radius. When no radius is specified, the radius defaults to 20 arcmin.
Coordinates should be an Astropy `SkyCoord()` object and the radius a Astropy `Quantity()`.

In [7]:
coord = coordinates.SkyCoord("18h53m35s +33d01m45s", frame='icrs')
radius = 5 * u.arcminute

result_table = Simbad.query_region(coord, radius=radius)
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 57,18 53 35.0967,+33 01 44.883
1,UCAC3 247-143405,18 53 34.1995,+33 02 55.423
2,UCAC3 247-143401,18 53 33.8083,+33 02 54.923
3,UCAC2 43471160,18 53 34.2870,+33 03 14.297
4,GB6 J1853+3301,18 53 41,+33 03.0
5,1RXS J185326.3+330027,18 53 26.302,+33 00 27.00
6,LEDA 2813726,18 53 46.0,+32 58 53
7,IC 1296,18 53 18.8159,+33 03 59.894


### Query by catalog

You can query all objects in a catalog with `query_catalog`. 
This can return a very large table so you should set a row limit on your query.
When querying large catalogs query by criteria is better.

In [6]:
Simbad_ = Simbad()  # define a new instance of the class
Simbad_.ROW_LIMIT = 12  # Set a row limit on that instance.
result_table = Simbad_.query_catalog('ngc')
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,NGC 1,00 07 15.8562,+27 42 29.020
1,NGC 2,00 07 17.1156,+27 40 42.034
2,NGC 3,00 07 16.7979,+08 18 05.976
3,NGC 4,00 07 24.4,+08 22 30
4,NGC 5,00 07 48.8659,+35 21 44.143
5,NGC 7,00 08 20.96,-29 54 54.0
6,NGC 8,00 08 45.3,+23 50 19
7,NGC 9,00 08 54.6826,+23 49 00.823
8,NGC 10,00 08 34.5389,-33 51 30.197


### Query by criteria

You can execute very specific queries using the `query_criteria()` method. These queries are formed in much the same way as they are on the web interface.
Descriptions of [queryable fields](https://simbad.cds.unistra.fr/simbad/sim-fsam) and [object types](https://simbad.cds.unistra.fr/guide/otypes.htx)
are on the SIMBAD website.

Below is an example query for variable stars near the Trapezium.

In [17]:
result_table = Simbad_.query_criteria('region(CIRCLE, Trapezium, 6m)', otype='V*')  # Using row limited instance.
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,V* KR Ori,05 35 00.1168,-05 23 01.884
1,* tet01 Ori D,05 35 17.2574,-05 23 16.570
2,* tet01 Ori E,05 35 15.7721,-05 23 09.889
3,* tet01 Ori F,05 35 16.7270,-05 23 25.197
4,* tet01 Ori A,05 35 15.8254,-05 23 14.334
5,V* MR Ori,05 35 16.9783,-05 21 45.337
6,V* AK Ori,05 35 26.3491,-05 25 40.220
7,V* V420 Ori,05 35 28.2055,-05 24 58.186
8,V* V404 Ori,05 35 03.0920,-05 22 37.827


### Query a bibcode

You can retrieve the references for a bibcode with `query_bibcode()`.

 example: [Ritchey, G.W.](https://ui.adsabs.harvard.edu/abs/1910ApJ....32...26R/abstract)

In [33]:
bibliography = Simbad.query_bibcode('1910ApJ....32...26R')
print(bibliography['References'][0])

1910ApJ....32...26R = DOI 10.1086/141784
Astrophys. J., 32, 26-35 (1910)
RITCHEY G.W.
On some methods and results in direct photography with the 60-inch reflecting telescope of the Mount Wilson Solar Observatory.


And get the objects referenced in the publication with `query_bibobj()`.

In [34]:
result_table = Simbad.query_bibobj('1910ApJ....32...26R')
result_table['MAIN_ID', 'RA', 'DEC'].show_in_notebook()

idx,MAIN_ID,RA,DEC
,,"""h:m:s""","""d:m:s"""
0,M 1,05 34 30.9,+22 00 53
1,M 81,09 55 33.1726,+69 03 55.062
2,M 97,11 14 47.7122,+55 01 08.482
3,MCG+09-19-014,11 14 58.373,+54 57 41.22
4,M 51,13 29 52.698,+47 11 42.93
5,NGC 5195,13 29 59.590,+47 15 58.06
6,M 57,18 53 35.0967,+33 01 44.883


## VizieR Queries